In [ ]:
# Import necessari
import cv2
import numpy as np
import os
import pickle
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from sklearn.model_selection import train_test_split
import json
import torchvision
from torchvision.models.detection import maskrcnn_resnet50_fpn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

In [ ]:
import json
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Load the JSON data
with open('Dataset_combined/588786395.json', 'r') as f:
    data = json.load(f)

# Extract the shapes from the JSON data
shapes = data['shapes']

# Create a figure and axis
fig, ax = plt.subplots()

# Set the aspect ratio to maintain the original aspect ratio of the image
ax.set_aspect('equal')

# Invert y-axis
ax.invert_yaxis()

# Loop through the shapes and plot them
for shape in shapes:
    label = shape['label']
    points = shape['points']
    
    # Create a polygon patch for the shape
    polygon = patches.Polygon(points, closed=True, fill=False, linewidth=2, edgecolor='r')
    ax.add_patch(polygon)

# Set the axis limits to fit the shapes
ax.relim()
ax.autoscale_view()

x_coords = [point[0] for shape in shapes for point in shape['points']]
y_coords = [point[1] for shape in shapes for point in shape['points']]
mask_width = max(x_coords) - min(x_coords)
mask_height = max(y_coords) - min(y_coords)
print('mask width', mask_height)
print('mask height', mask_width)

# Show the plot
plt.show()

import matplotlib.image as mpimg

# Load and plot an image
img = mpimg.imread('Dataset/588786395_training.png')  # Sostituisci 'path_to_your_image.png' con il percorso dell'immagine PNG
plt.imshow(img)
plt.axis('off')  # Nasconde gli assi
height, width, _ = img.shape

print("Dimensioni dell'immagine:")
print("Altezza:", height)
print("Larghezza:", width)
plt.show()

from PIL import Image

# Carica l'immagine
img = Image.open('Dataset/588786395_training.png')  # Sostituisci 'path_to_your_image.png' con il percorso dell'immagine PNG

# Scalare l'immagine alle nuove dimensioni
new_width = 440
new_height = height+(mask_width-width)
print(new_width)
print(new_height)
resized_img = img.resize((int(new_width), int(new_height)))

plt.imshow(resized_img)
plt.axis('off')
plt.show()


In [ ]:
import os
import shutil

# Define the paths to the mask and image folders
mask_folder = 'Dataset_mask'
image_folder = 'Dataset_edge'

# Create a new folder to store the combined images and masks
combined_folder = 'Dataset_combined'
os.makedirs(combined_folder, exist_ok=True)

# Copy all files from the mask folder to the combined folder
for mask_file in os.listdir(mask_folder):
    shutil.copy2(os.path.join(mask_folder, mask_file), combined_folder)

# Copy all files from the image folder to the combined folder
for image_file in os.listdir(image_folder):
    shutil.copy2(os.path.join(image_folder, image_file), combined_folder)

print("Masks and images have been combined and stored in the 'Combined' folder.")

In [ ]:
import json
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

# Load the JSON data
with open('Dataset_combined/588786395.json', 'r') as f:
    data = json.load(f)

# Extract the shapes from the JSON data
shapes = data['shapes']

# Create a figure and axis
fig, ax = plt.subplots()

# Set the aspect ratio to maintain the original aspect ratio of the image
ax.set_aspect('equal')

# Invert y-axis
ax.invert_yaxis()

# Loop through the shapes and plot them
for shape in shapes:
    label = shape['label']
    points = shape['points']
    
    # Create a polygon patch for the shape
    polygon = patches.Polygon(points, closed=True, fill=False, linewidth=2, edgecolor='r')
    ax.add_patch(polygon)

# Set the axis limits to fit the shapes
ax.relim()
ax.autoscale_view()

from PIL import Image

# Carica l'immagine originale
img = Image.open('Dataset/588786395_training.png')  # Sostituisci 'path_to_your_image.png' con il percorso dell'immagine originale

# Altezza desiderata per la nuova immagine
new_height = 1024

# Calcola la nuova larghezza mantenendo il rapporto di aspetto
width, height = img.size
aspect_ratio = width / height
new_width = int(aspect_ratio * new_height)
print(new_height, new_width)

# Ridimensiona l'immagine
resized_img = img.resize((new_width, new_height))

# Salva o mostra l'immagine ridimensionata
#resized_img.show()
#resized_img.save('path_to_save_resized_image.png')  # Sostituisci 'path_to_save_resized_image.png' con il percorso in cui desideri salvare l'immagine ridimensionata


# Load the original image
#original_image = Image.open('Dataset/387805419_training.png')  # Sostituisci 'path_to_your_original_image.png' con il percorso dell'immagine originale
# Resize the original image
#new_width = 440
#new_height = 468
#resized_image = original_image.resize((new_width, new_height))

# Show the resized image
plt.imshow(resized_img)

# Show the plot
plt.axis('off')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Carica le immagini
img1 = mpimg.imread('Dataset/588786395_training.png')
img2 = mpimg.imread(r'C:\Users\loren\Downloads\APKLOT-master\APKLOT-master\1. Satellite\Dataset\parcheggi\segmented_images\588786395_SegmentationClass.png')

# Imposta il grado di trasparenza per ciascuna immagine
alpha = 1.0  # Modifica questo valore per controllare il grado di trasparenza

# Sovrapponi le due immagini
blended_img = alpha * img1 + (1 - alpha) * img2

# Plotta l'immagine blendata
plt.imshow(blended_img)
plt.axis('off')  # Nascondi gli assi
plt.show()


In [ ]:
import os
import glob
from PIL import Image
import os
import glob
from PIL import Image

class ParkingLotDataset(Dataset):
    def __init__(self, root_img, root_msk, pairs=None, transforms=None):
        self.root_img = root_img
        self.root_msk = root_msk
        self.transforms = transforms

        if pairs is None:
            # Get all image files
            self.image_paths = sorted(glob.glob(os.path.join(root_img, '*.png')))

            # Get all mask files
            self.mask_paths = sorted(glob.glob(os.path.join(root_msk, '*.png')))

            # Pair image and mask files based on their filenames
            #self.pairs = [(image_path, mask_path) for image_path in self.image_paths for mask_path in self.mask_paths if os.path.splitext(os.path.basename(image_path))[0] == os.path.splitext(os.path.basename(mask_path))[0]]
            self.pairs = []

            for image_path in self.image_paths:
                image_filename = os.path.splitext(os.path.basename(image_path))[0]
                for mask_path in self.mask_paths:
                    mask_filename = os.path.splitext(os.path.basename(mask_path))[0]
                    if image_filename in mask_filename:
                        self.pairs.append((image_path, mask_path))
                        break  # Se trova una corrispondenza, passa al prossimo file immagine
        else:
            self.pairs = pairs

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        image_path, mask_path = self.pairs[idx]

        # Load image
        image = Image.open(image_path)
        image_array = np.array(image)

        # Apply transformations
        if self.transforms:
            image_array = self.transforms(image_array)
            print(image_array.shape)
        

    


        return image_array, masks
        

        #merged_mask = np.zeros((2108, 2113), dtype=np.uint8)

        # Load and plot the masks
        #with open(mask_path, 'r') as f:
        #    masks_data = json.load(f)

        #for shape in masks_data['shapes']:
        #    points = np.array(shape['points'])
        #    mask = np.zeros((2108, 2113), dtype=np.uint8)
        #    cv2.fillPoly(mask, [points.astype(np.int32)], 1)
        #    _, binary_mask = cv2.threshold(mask, 0, 1, cv2.THRESH_BINARY)
        #    merged_mask += binary_mask

        #if self.transforms:
        #    masks = self.transforms(merged_mask)
        #    print(masks.shape)

        #print("Image shape:", image_array.shape)
        #print("Masks shape:", masks.shape)

plotting 1

In [ ]:
import matplotlib.pyplot as plt

# Assuming you have a DataLoader object called train_loader
num_samples = 10

# Iterate over batches from the DataLoader
for batch_idx, (images, masks) in enumerate(train_loader):
    # Get the number of samples in the current batch
    batch_size = images.size(0)

    # Iterate over the samples in the batch
    for i in range(batch_size):
        # Convert the image tensor to a NumPy array
        image_array = images[i].permute(1, 2, 0).numpy()

        # Convert the mask tensor to a NumPy array
        mask_array = masks[i].squeeze().numpy()

        # Plot the image and the combined mask
        plt.figure(figsize=(12, 6))
        plt.subplot(1, 2, 1)
        plt.imshow(image_array, cmap='gray')
        plt.title(f'Image {batch_idx * batch_size + i + 1}')
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.imshow(image_array, cmap='gray')
        plt.imshow(mask_array, cmap='gray', alpha=0.5)
        plt.title(f'Image {batch_idx * batch_size + i + 1} with Masks')
        plt.axis('off')

        plt.show()

        # Break after the first 10 samples
        if batch_idx * batch_size + i == num_samples - 1:
            break

    # Break after processing one batch
    if batch_idx == 0:
        break

plotting 2

In [ ]:
import matplotlib.pyplot as plt

# Get the first pair of image and masks from the dataset
image_path, mask_path = train_dataset.pairs[0]
print(image_path)
print(mask_path)
print(train_dataset.pairs[0])

# Load the image
image = Image.open(image_path)
image_array = np.array(image)

# Plot the image
plt.figure(figsize=(12, 6))
plt.imshow(image_array, cmap='gray')
plt.title('Image')

merged_mask = np.zeros((2108, 2113), dtype=np.uint8)

# Load and plot the masks
with open(mask_path, 'r') as f:
    masks_data = json.load(f)

for i, shape in enumerate(masks_data['shapes']):
    points = np.array(shape['points'])
    mask = np.zeros((2108, 2113), dtype=np.uint8)
    cv2.fillPoly(mask, [points.astype(np.int32)], 1)
    _, binary_mask = cv2.threshold(mask, 0, 1, cv2.THRESH_BINARY)
    merged_mask += binary_mask
plt.imshow(merged_mask, cmap='gray', alpha=0.0)
plt.title(f'Image with Masks')
plt.show()


In [ ]:

        #merged_mask = np.zeros((2108, 2113), dtype=np.uint8)

        # Load and plot the masks
        #with open(mask_path, 'r') as f:
        #    masks_data = json.load(f)

        #for shape in masks_data['shapes']:
        #    points = np.array(shape['points'])
        #    mask = np.zeros((2108, 2113), dtype=np.uint8)
        #    cv2.fillPoly(mask, [points.astype(np.int32)], 1)
        #    _, binary_mask = cv2.threshold(mask, 0, 1, cv2.THRESH_BINARY)
        #    merged_mask += binary_mask

        #if self.transforms:
        #    masks = self.transforms(merged_mask)
        #    print(masks.shape)

        #print("Image shape:", image_array.shape)
        #print("Masks shape:", masks.shape)

CALCOLATORE DI DIMENSIONI IMMAGINI

In [24]:
#controllo il massimo valore di altezza e larghezza per fare il padding
import cv2
import os

# Directory containing the images
data_dir = 'Dataset_edge_smaller'

# Initialize variables to store max height and width
max_height = 0
max_width = 0

# Iterate through all images in the directory
for filename in os.listdir(data_dir):
    if filename.endswith('.png') or filename.endswith('.jpg'):
        # Load the image
        image_path = os.path.join(data_dir, filename)
        image = cv2.imread(image_path)

        # Get height and width of the image
        height, width, _ = image.shape
        if(height > 500 or width > 500):
            print(filename)
            print(image.shape)

        # Update max_height and max_width if necessary
        if height > max_height:
            max_height = height
        if width > max_width:
            max_width = width

        

#Image with different max_height and max_width: Queretaro-TEC_parcheggi.png
print("Maximum Height:", max_height)
print("Maximum Width:", max_width)

101881731.png
(512, 512, 3)
103109948.png
(512, 512, 3)
103614205.png
(512, 512, 3)
104101927.png
(512, 512, 3)
104357814.png
(512, 512, 3)
104357815.png
(512, 512, 3)
104357817.png
(512, 512, 3)
104907575.png
(512, 512, 3)
116016718.png
(512, 512, 3)
119422952.png
(512, 512, 3)
119917133.png
(512, 512, 3)
126381586.png
(512, 512, 3)
126381588.png
(512, 512, 3)
126381594.png
(512, 512, 3)
127649594.png
(512, 512, 3)
127649606.png
(512, 512, 3)
130730813.png
(512, 512, 3)
132613254.png
(512, 512, 3)
134714700.png
(512, 512, 3)
136128142.png
(512, 512, 3)
136128143.png
(512, 512, 3)
136128144.png
(512, 512, 3)
137007454.png
(512, 512, 3)
137007455.png
(512, 512, 3)
139146552.png
(512, 512, 3)
139146568.png
(512, 512, 3)
144384815.png
(512, 512, 3)
144848258.png
(512, 512, 3)
147397541.png
(512, 512, 3)
148521975.png
(512, 512, 3)
148788621.png
(512, 512, 3)
151179182.png
(512, 512, 3)
152582726.png
(512, 512, 3)
153028544.png
(512, 512, 3)
15309700.png
(512, 512, 3)
15311494.png
(512, 51

Image resize

In [19]:
import cv2
import os

# Directory containing the images
data_dir = 'resized_masks512'
# Directory to save resized images
resized_dir = 'resized_masks512_bis'

# Ensure the resized_dir exists
os.makedirs(resized_dir, exist_ok=True)

# Iterate through all images in the directory
for filename in os.listdir(data_dir):
    if filename.endswith('.png') or filename.endswith('.jpg'):
        # Load the image
        image_path = os.path.join(data_dir, filename)
        image = cv2.imread(image_path)

        # Get height and width of the image
        height, width, _ = image.shape

        # If the image is larger than 1000 in either dimension, resize it
        if height > 512 or width > 512:
            # Calculate the scale factor
            scale_factor = 0.5  # Reduce size by half

            # Calculate the new dimensions of the image
            new_width = int(width * scale_factor)
            new_height = int(height * scale_factor)

            # Resize the image
            image = cv2.resize(image, (new_width, new_height))

        # Save the image (resized or original) to the new directory
        new_image_path = os.path.join(resized_dir, filename)
        cv2.imwrite(new_image_path, image)
        

Mask resize and padding

In [21]:
import cv2
import os

# Directory containing the images
data_dir = 'Resized_masks512'
# Directory to save resized images
resized_dir = 'Resizedmasks_smaller'

# Ensure the resized_dir exists
os.makedirs(resized_dir, exist_ok=True)

# Iterate through all images in the directory
for filename in os.listdir(data_dir):
    if filename.endswith('.png') or filename.endswith('.jpg'):
        # Load the image
        image_path = os.path.join(data_dir, filename)
        image = cv2.imread(image_path)

        # Get height and width of the image
        height, width, _ = image.shape

        # If the image is larger than 1000 in either dimension, resize it
        if height > 512 or width > 512:
            # Calculate the scale factor
            scale_factor = 0.5  # Reduce size by half

            # Calculate the new dimensions of the image
            new_width = int(width * scale_factor)
            new_height = int(height * scale_factor)

            # Resize the image
            image = cv2.resize(image, (new_width, new_height))

        ## MASK PADDING
        max_height = 512
        max_width = 512
        padding_height = max(0, max_height - image.shape[0])
        padding_width = max(0, max_width - image.shape[1])
        mask_padded = cv2.copyMakeBorder(image, 0, padding_height, 0, padding_width, cv2.BORDER_CONSTANT, value=0)
        # Save the image (resized or original) to the new directory
        new_image_path = os.path.join(resized_dir, filename)
        cv2.imwrite(new_image_path, mask_padded)


Dimension checker

In [ ]:
for filename in os.listdir('Dataset_edge'):
    image_path = os.path.join('Dataset_edge', filename)
    image = cv2.imread(image_path)
    if image.shape[0] != 1056 or image.shape[1] != 1056:
        print(image_path)
        print(image.shape[0],image.shape[1])

Store canny edge

In [23]:
dataset_path = 'ResizedDataset_even_smaller'
edge_path = 'Dataset_edge_smaller'

os.makedirs(edge_path, exist_ok=True)

def get_edge_image(image, low_threshold, high_threshold):
    # Convert the image to grayscale if it's not already
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Apply Canny edge detection
    edges = cv2.Canny(gray, low_threshold, high_threshold)
    
    return edges
#edge detection + padding immagini
for filename in os.listdir(dataset_path):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        
        input_image_path = os.path.join(dataset_path, filename)
        image = cv2.imread(input_image_path)
        # Ottieni l'immagine del bordo
        edge_image = get_edge_image(image, low_threshold=200, high_threshold=300)
        
        # Apply padding to the edge image
        max_height = 512
        max_width = 512
        padding_height = max(0, max_height - edge_image.shape[0])
        padding_width = max(0, max_width - edge_image.shape[1])
        edge_image_padded = cv2.copyMakeBorder(edge_image, 0, padding_height, 0, padding_width, cv2.BORDER_CONSTANT, value=0)
        #Verificare se il nome del file contiene "_testing" o "_training"
        if "_testing" in filename or "_training" in filename:
            # Rinominare il file rimuovendo "_testing" o "_training" dal nome
            new_filename = filename.replace("_testing", "").replace("_training", "")
            
            print(f"Rinominato il file {filename} in {new_filename}")
            # Salva l'immagine del bordo nella cartella di output
            output_image_path = os.path.join(edge_path, new_filename)
            cv2.imwrite(output_image_path, edge_image_padded)

print("Il rilevamento dei bordi è stato applicato a tutte le immagini nella cartella.")


Rinominato il file 101881731_training.png in 101881731.png
Rinominato il file 103109948_testing.png in 103109948.png
Rinominato il file 103614205_training.png in 103614205.png
Rinominato il file 104101927_testing.png in 104101927.png
Rinominato il file 104357814_training.png in 104357814.png
Rinominato il file 104357815_training.png in 104357815.png
Rinominato il file 104357817_training.png in 104357817.png
Rinominato il file 104907575_testing.png in 104907575.png
Rinominato il file 116016718_training.png in 116016718.png
Rinominato il file 119422952_training.png in 119422952.png
Rinominato il file 119917133_training.png in 119917133.png
Rinominato il file 126381586_training.png in 126381586.png
Rinominato il file 126381588_training.png in 126381588.png
Rinominato il file 126381594_training.png in 126381594.png
Rinominato il file 127649594_training.png in 127649594.png
Rinominato il file 127649606_testing.png in 127649606.png
Rinominato il file 130730813_training.png in 130730813.png
R